In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QTextEdit
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
import rasterio

class GeoTIFFLabel(QLabel):
    def __init__(self, coordinate_display):
        super().__init__()
        self.image_transform = None
        self.coordinate_display = coordinate_display

    def mouseMoveEvent(self, event):
        if self.image_transform:
            x = event.x()
            y = event.y()
            # Convert mouse coordinates to geographic coordinates
            lon, lat = self.image_transform * (x, y)
            # Update coordinate display
            self.coordinate_display.setText(f"Mouse Position (x, y): ({x}, {y})\n"
                                             f"Latitude: {lat}, Longitude: {lon}")

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("GeoTIFF Viewer")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        layout = QVBoxLayout()
        self.central_widget.setLayout(layout)

        # Text box to display coordinates
        self.coordinate_display = QTextEdit()
        self.coordinate_display.setReadOnly(True)
        layout.addWidget(self.coordinate_display)

        # Load GeoTIFF
        self.image_label = GeoTIFFLabel(self.coordinate_display)
        layout.addWidget(self.image_label)

        self.load_geotiff("testGeoTiff.tif")

    def load_geotiff(self, file_path):
        # Open GeoTIFF
        try:
            dataset = rasterio.open(file_path)
        except Exception as e:
            print(f"Failed to open the GeoTIFF file: {e}")
            return

        # Read raster data
        data = dataset.read(1)  # Assuming single band
        width = dataset.width
        height = dataset.height

        # Normalize data to 8-bit for displaying
        normalized_data = (data / data.max() * 255).astype('uint8')

        # Get transformation
        transform = dataset.transform

        # Set image transform
        self.image_label.image_transform = transform

        # Convert to QImage
        image = QImage(normalized_data, width, height, width, QImage.Format_Grayscale8)

        # Display image
        pixmap = QPixmap.fromImage(image)
        self.image_label.setPixmap(pixmap)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QVBoxLayout, QWidget, QTextEdit, QHBoxLayout, QSizePolicy
from PyQt5.QtGui import QPixmap, QImage, QFont, QPainter, QTransform
from PyQt5.QtCore import Qt, QPoint
import rasterio

class GeoTIFFLabel(QLabel):
    def __init__(self, coordinate_displays):
        super().__init__()
        self.image_transform = None
        self.coordinate_displays = coordinate_displays
        self.zoom_factor = 1.0
        self.pan_offset = QPoint(0, 0)

    def wheelEvent(self, event):
        zoom_speed = 0.1  # Adjust zoom speed as needed
        # Zoom in or out based on wheel event
        if event.angleDelta().y() > 0:
            self.zoom_factor += zoom_speed
        else:
            self.zoom_factor -= zoom_speed
            if self.zoom_factor < 0.1:  # Limit zoom out level
                self.zoom_factor = 0.1
        self.update()

    def mousePressEvent(self, event):
        if event.buttons() == Qt.LeftButton:
            self.last_pan_pos = event.pos()

    def mouseMoveEvent(self, event):
        if event.buttons() == Qt.LeftButton:
            pan_delta = event.pos() - self.last_pan_pos
            self.last_pan_pos = event.pos()
            self.pan_offset += pan_delta
            self.update()

    def paintEvent(self, event):
        if self.pixmap() is not None:
            painter = QPainter(self)
            painter.setRenderHint(QPainter.SmoothPixmapTransform)
            painter.setTransform(QTransform().translate(self.pan_offset.x(), self.pan_offset.y()).scale(self.zoom_factor, self.zoom_factor))
            painter.drawPixmap(0, 0, self.pixmap())

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("GeoTIFF Viewer")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        layout = QVBoxLayout()
        self.central_widget.setLayout(layout)

        # Create a layout for displaying text boxes in one line
        hbox_layout = QHBoxLayout()

        # Create text boxes for displaying coordinates
        self.coordinate_displays = []
    

        # Load GeoTIFF
        self.image_label = GeoTIFFLabel(self.coordinate_displays)
        layout.addWidget(self.image_label)
        self.load_geotiff("testGeoTiff.tif")

    def load_geotiff(self, file_path):
        # Open GeoTIFF
        try:
            dataset = rasterio.open(file_path)
        except Exception as e:
            print(f"Failed to open the GeoTIFF file: {e}")
            return

        # Read raster data
        data = dataset.read(1)  # Assuming single band
        width = dataset.width
        height = dataset.height

        # Normalize data to 8-bit for displaying
        normalized_data = (data / data.max() * 255).astype('uint8')

        # Get transformation
        transform = dataset.transform

        # Set image transform
        self.image_label.image_transform = transform

        # Display image
        image = QImage(normalized_data, width, height, width, QImage.Format_Grayscale8)
        pixmap = QPixmap.fromImage(image)
        self.image_label.setPixmap(pixmap)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())
